## Traitement de Données Massives

# Projet Partie n°1 

### Binôme : Sapy Oscar & Berthillon Mickaël

# 1 : Collecte de données

In [8]:
import requests
import os

# Define the API endpoint and your access key
url = "https://api.pexels.com/v1/search"
access_key = "MgxBjp4CnSFtTvWe3SeHtkTGp5oUOBWVG5S0PYZAd37hV3T3h2yY4PHp"

# Define the headers with your access key
headers = {
    "Authorization": access_key
}

# Define the path to the images folder
path = os.path.abspath("./images")

# Define the number of images to download
num_images = 15

# Define the query parameters
query_params = {
    "query": "random", 
    "per_page": num_images
}

# Create the images folder if it doesn't exist
if not os.path.exists(path):
    os.makedirs(path)

# Calculate the number of missing images
existing_images = len(os.listdir(path))
missing_images = max(num_images - existing_images, 0)

# Update the query parameters to request the missing images
query_params["per_page"] = missing_images

# Send the API request and download the images if there are any missing
if missing_images > 0:
    response = requests.get(url, headers=headers, params=query_params)
    json = response.json()
    for i in range(missing_images):
        image_url = json["photos"][i]["src"]["original"]
        image_id = json["photos"][i]["id"]
        image_extension = ".jpg" # change extension as per your requirement
        image_filename = f"{image_id}{image_extension}"
        image_path = os.path.join(path, image_filename)

        # Download the image and save it to the images folder
        response = requests.get(image_url)
        with open(image_path, "wb") as f:
            f.write(response.content)


# 2 : Etiquetage et annotation

In [9]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS
from PIL.TiffImagePlugin import IFDRational

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            try:
                return obj.decode('utf-8')
            except UnicodeDecodeError:
                return obj.decode('utf-8', 'replace')
        elif isinstance(obj, IFDRational):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

# Path to the folder containing pictures
path_to_folder = os.path.abspath('./images')

# Create the metadata folder if it doesn't exist
if not os.path.exists('metadata'):
    os.mkdir('metadata')

# Loop over all pictures in the folder
for filename in os.listdir(path_to_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Open the image and get the EXIF data
        image = Image.open(os.path.join(path_to_folder, filename))
        exifdata = image.getexif()

        # Create a dictionary to store the metadata for this image
        metadata = {}

        # Loop over all EXIF tags and add them to the metadata dictionary
        for tag_id, value in exifdata.items():
            tag = TAGS.get(tag_id, tag_id)
            metadata[tag] = value

        # Write the metadata dictionary to a JSON file in the metadata folder
        json_filename = os.path.splitext(filename)[0] + '.json'
        with open(os.path.join('metadata', json_filename), 'w') as f:
            json.dump(metadata, f, cls=MyEncoder)


/tmp/ipykernel_878/3212664523.py:29: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/3844786.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_878/3212664523.py:29: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/4226881.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_878/3212664523.py:29: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/7004697.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_878/3212664523.py:29: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/4022082.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_878/3212664523.py:29: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/im

# 3 : Analyse de données

In [7]:
import os
import json
import cv2
from PIL import Image
from colorthief import ColorThief
import pytesseract
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T

# Initialize torchvision object detection model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

image_folder = os.path.abspath("./images")
metadata_folder = os.path.abspath("./metadata")

if not os.path.exists(metadata_folder):
    os.makedirs(metadata_folder)

def check_existing_metadata(metadata):
    for file in os.listdir(metadata_folder):
        file_path = os.path.join(metadata_folder, file)
        with open(file_path, 'r') as f:
            existing_metadata = json.load(f)
            if existing_metadata == metadata:
                return True
    return False

# Function to get image metadata
def get_image_metadata(image_path):
    # ... (metadata extraction code remains the same) ...

    # Loop over all pictures in the folder
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            metadata_file = os.path.join(metadata_folder, os.path.splitext(filename)[0] + ".json")

            # Check if metadata file already exists
            if not os.path.exists(metadata_file):
                # Get the image metadata
                image_path = os.path.join(image_folder, filename)
                metadata = get_image_metadata(image_path)

                # If the metadata dictionary is already present in any of the existing metadata files, skip this image
                if check_existing_metadata(metadata):
                    continue

                # Write the metadata dictionary to a JSON file in the metadata folder
                with open(metadata_file, "w") as f:
                    json.dump(metadata, f)


/home/mick/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mick/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 4 : Visualisation de données

TypeError: expected str, bytes or os.PathLike object, not bool

In [22]:
import tkinter as tk
from PIL import Image, ImageTk
import os
import json
import random

def show_next_image():
    global current_image_index
    current_image_index += 1
    if current_image_index == len(selected_images):
        current_image_index = 0
    image_path = os.path.join(image_folder, selected_images[current_image_index])
    image = Image.open(image_path)
    image.thumbnail((400, 400))
    photo = ImageTk.PhotoImage(image)
    image_label.config(image=photo)
    image_label.image = photo

def like_image():
    global liked_count
    liked_count += 1
    current_image = selected_images[current_image_index]
    metadata_file = os.path.join(metadata_folder, os.path.splitext(current_image)[0] + ".json")
    with open(metadata_file, "r") as f:
        metadata = json.load(f)
    preferred_data.append(metadata)

    # Save the preferred_data in a JSON file
    with open(preferred_data_file, "w") as f:
        json.dump(preferred_data, f)

    if liked_count >= 5:
        show_best_image()
    else:
        show_next_image()

def show_best_image():
    global best_image_displayed
    if not best_image_displayed:
        best_image = random.choice(os.listdir(image_folder))
        image_path = os.path.join(image_folder, best_image)
        image = Image.open(image_path)
        image.thumbnail((400, 400))
        photo = ImageTk.PhotoImage(image)
        image_label.config(image=photo)
        image_label.image = photo
        best_image_displayed = True
        instructions.config(text="Here is the most accurate image for you:")

def quit_app():
    root.destroy()

image_folder = "images"
metadata_folder = "metadata"
preferred_data_file = "preferred_data.json"
selected_images = random.sample(os.listdir(image_folder), 5)
current_image_index = -1
liked_count = 0
best_image_displayed = False
preferred_data = []

root = tk.Tk()
instructions = tk.Label(root, text="Choose the images you like:")
instructions.pack()

image_label = tk.Label(root)
image_label.pack()

like_button = tk.Button(root, text="Like", command=like_image)
like_button.pack()

pass_button = tk.Button(root, text="Pass", command=show_next_image)
pass_button.pack()

quit_button = tk.Button(root, text="Quit", command=quit_app)
quit_button.pack()

show_next_image()
root.mainloop()
